In [27]:
import pandas as pd
import numpy as np

In [28]:
operation_types = ['System Specification','Architectural Design','Functional Design',
                    'Physical Design','Verification and Signoff','Fabrication','Packaging and Testing']

In [29]:
users = pd.read_csv(r'dataset/user.csv')

In [30]:
import string
import random

In [31]:
'''CREATE TABLE package
(USER_ID   INT,
PACKAGE_ID  INT,
BUDGET  DEC(6,2),
CREATE_TIME DATETIME,
DEADLINE  DATETIME,
PRIMARY KEY(PACKAGE_ID),
FOREIGN KEY(USER_ID) REFERENCES user(USER_ID));'''

user_id = np.random.choice([i for i in range(300)]+[i for i in range(700,850)],size=500)
package_id = [i for i in range(500)]
budget = np.random.normal(2000,700,size=1000)
budget = [i for i in budget if i>200][:500]


In [32]:
import datetime
import time
 
end_time=datetime.datetime.now()
start_time=datetime.datetime.now() + datetime.timedelta(days=-365) # 当前时间减去3分钟
 
a1=tuple(start_time.timetuple()[0:9])    #设置开始日期时间元组（2020-04-11 16:30:21）
a2=tuple(end_time.timetuple()[0:9])   #设置结束日期时间元组（2020-04-11 16:33:21）
 
start=time.mktime(a1)    #生成开始时间戳
end=time.mktime(a2)      #生成结束时间戳

create_time = []
deadline = []
#随机生成日期字符串
for i in range(len(user_id)):
    t=random.randint(start,end)    #在开始和结束时间戳中随机取出一个
    date_touple=time.localtime(t)          #将时间戳生成时间元组
    date=time.strftime("%Y-%m-%d %H:%M:%S",date_touple)   #将时间元组转成格式化字符串（2020-04-11 16:33:21）
    date = datetime.datetime.strptime(date,"%Y-%m-%d %H:%M:%S")
    create_time.append(date)

    duration = random.randint(10,20)
    ddl = date + datetime.timedelta(days=duration)
    # ddl = datetime.datetime.strptime(date,"%Y-%m-%d %H:%M:%S") + datetime.timedelta(days=duration)
    # ddl = time.strftime("%Y-%m-%d %H:%M:%S",ddl)
    deadline.append(ddl)

In [33]:
package = pd.DataFrame(data={'user_id':user_id,'package_id':package_id,
            'budget':budget,'create_time':create_time,'deadline':deadline})
package.to_csv('dataset/package.csv',index=False)

In [34]:
'''CREATE TABLE chip_type
(CHIP_NAME  VARCHAR(20),
CHIP_VERSION  VARCHAR(20),
PRIMARY KEY(CHIP_NAME,CHIP_VERSION));
'''
number_of_strings = 25
length_of_string = 5
chip_name=[]
for x in range(number_of_strings):
  name=''.join(random.SystemRandom().choice(string.ascii_letters + string.digits) for _ in range(length_of_string))
  chip_name.append(name)
chip_name += chip_name[:10]
chip_name += chip_name[:5]
versions = [1]*25+[2]*10+[3]*5
chip_type = pd.DataFrame(data={'chip_name':chip_name, 'chip_version':versions})
chip_type = chip_type.sort_values(by=['chip_name','chip_version'],ignore_index=True)
chip_type.to_csv('dataset/chip_type.csv',index=False)

In [35]:
'''CREATE TABLE chip
(CHIP_NAME  VARCHAR(20),
CHIP_VERSION  VARCHAR(20),
PACKAGE_ID  INT,
CHIP_ID  INT,
FOREIGN KEY(CHIP_NAME,CHIP_VERSION) REFERENCES chip_type(CHIP_NAME,CHIP_VERSION),
FOREIGN KEY(PACKAGE_ID) REFERENCES package(PACKAGE_ID));'''
package_id = np.random.choice([i for i in range(150)]+[i for i in range(150,500)],
                                size=800,p=[0.5/150]*150+[0.5/350]*350)
chip = chip_type.sample(n=800, random_state=0,replace=True)
chip_id = [i for i in range(800)]
chip['package_id'],chip['chip_id']=package_id,chip_id
chip.reset_index(inplace=True,drop=True)
chip.to_csv('dataset/chip.csv',index=False)

In [36]:
'''CREATE TABLE state
(PACKAGE_ID  INT,
START_TIME DATETIME,
END_TIME DATETIME,
NAME VARCHAR(20),
FOREIGN KEY(PACKAGE_ID) REFERENCES package(PACKAGE_ID));'''

start_time1, end_time1 = [], []
for i in range(len(package)):
    ct =  create_time[i]
    lag = random.randint(1,8)
    st = ct + datetime.timedelta(hours=lag)
    start_time1.append(st)

    ddl = deadline[i]
    lag2 = random.randint(-100,0)
    et = ddl + datetime.timedelta(hours=lag2)
    end_time1.append(et)
state = pd.DataFrame(data={'package_id':package['package_id'],'start_time':start_time1,'end_time':end_time1})
state.to_csv('dataset/state.csv',index=False)